# Benchmark on all dataset

In [1]:
# Benchmark all models produced in this study
from models.deepprime import predict as deepprime_predict
from models.pridict import predict as pridict_predict
from models.transformer import predict as transformer_predict
from models.ensemble_adaboost import predict as ensemble_adaboost_predict
from models.ensemble_bagging import predict as ensemble_bagging_predict
from models.ensemble_weighted_mean import predict as ensemble_weighted_mean_predict
from models.conventional_ml_models import mlp_predict, random_forest_predict, xgboost_predict, ridge_predict
from glob import glob
from os.path import join as pjoin, basename

# run predictions on all models on each dataset, calculate the pearson and spearman correlation
performance = {}

for data in glob(pjoin('models', 'data', 'std', '*.csv')):
    data_name = basename(data)
    print(data_name)
    # run predictions on all models
    for model in [deepprime_predict, pridict_predict, transformer_predict, ensemble_adaboost_predict, ensemble_bagging_predict, ensemble_weighted_mean_predict, mlp_predict, random_forest_predict, xgboost_predict, ridge_predict]:
        print(model)
        predictions = model(data_name)

# Benchmark on Individual Edit Types